# Proyecto Rifeco GG: Clasificación de jugadores por liga.
### Data Mining 2021 UTFSM
#### Integrantes:
* Matías Concha 201530017-8
* Jorge Fernández 2015 xd
* María Fernanda Rivas 201584033-4

## Introducción

### Objetivo
El objetivo de este proyecto es poder clasificar de manera correcta las ligas de los jugadores en LAS que es la región de america del sur y asi, poder saber si los puntajes obtenidos por los jugadores en sus partidas son suficiente para poder clasificarlos desde la mejor liga hasta la peor liga.
Además también se compararán 2 regiones importantes en el juego, las cuales son Corea y Brasil para saber si es posible clasificar por región y si los jugadores de estas regiones tienen puntajes demasiado distinto entre ellos.

# Bibliotecas

In [59]:
# Python basic libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from matplotlib.colors import ListedColormap

In [60]:
# Data Preprocessing libraries

# for splitting data into training and testing
from sklearn.model_selection import train_test_split

# standardize features removing the mean and scaling to unit variance
from sklearn.preprocessing import StandardScaler

# metris for classifiers evaluation
from sklearn import metrics

from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split

from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report



In [61]:
# Classifiers
from sklearn.neural_network import MLPClassifier # multilayer perceptron classifier
from sklearn.neighbors import KNeighborsClassifier # K nearest neighbors classifier
from sklearn.svm import SVC # Support vector machine, C-support vector classifier
from sklearn.gaussian_process import GaussianProcessClassifier # Gaussian process classifier
from sklearn.gaussian_process.kernels import RBF # radial basis function kernel (squared exponential kernel)
from sklearn.tree import DecisionTreeClassifier # Decision tree classifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier # Ensemble classifiers: Random forest and AdaBoost
from sklearn.naive_bayes import GaussianNB # Naive Bayes classifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis # Quadratic Discriminant Analysis classifier

# Adquisición de datos

In [62]:
# Reading the data
df = pd.read_csv('csv/kr_br1_50matchs.csv')
df.describe()

,gameId,wins,losses,championId,spell1Id,spell2Id,kills,deaths,assists,largestKillingSpree,...,totalMinionsKilled,goldEarned,goldSpent,visionScore,team-towerKills,team-inhibitorKills,team-baronKills,team-dragonKills,team-vilemawKills,team-riftHeraldKills
count,5.872900e+04,58729.000000,58729.000000,58729.000000,58729.000000,58729.000000,58729.000000,58729.000000,58729.000000,58729.000000,...,58729.000000,58729.000000,58729.000000,58729.000000,58729.000000,58729.000000,58729.000000,58729.000000,58729.000000,58729.000000
mean,3.750759e+09,100.358000,100.156107,161.007390,10.479065,8.872482,6.500094,6.430741,8.668954,2.797698,...,93.207717,10852.840607,9971.536924,17.470756,4.637879,0.815338,0.314683,1.536055,0.000034,0.583494
std,1.493748e+09,123.882709,119.137391,190.794968,10.118696,9.890312,5.472639,3.701050,7.190802,2.925078,...,69.776045,4312.708146,4361.719386,16.377203,3.595477,1.106186,0.570129,1.412020,0.005836,0.705574
min,1.712007e+09,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,667.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.305645e+09,17.000000,21.000000,48.000000,4.000000,4.000000,2.000000,4.000000,3.000000,0.000000,...,31.000000,7843.000000,7050.000000,7.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2.323233e+09,55.000000,59.000000,96.000000,7.000000,4.000000,5.000000,6.000000,7.000000,2.000000,...,80.000000,10554.000000,9650.000000,14.000000,4.000000,0.000000,0.000000,1.000000,0.000000,0.000000
75%,5.314981e+09,139.000000,133.000000,202.000000,14.000000,12.000000,9.000000,9.000000,12.000000,4.000000,...,147.000000,13518.000000,12500.000000,24.000000,8.000000,1.000000,1.000000,3.000000,0.000000,1.000000
max,5.361712e+09,1082.000000,1115.000000,887.000000,54.000000,54.000000,75.000000,28.000000,64.000000,75.000000,...,398.000000,38957.000000,238645.000000,166.000000,11.000000,11.000000,4.000000,7.000000,1.000000,2.000000


In [63]:
# Review missing data
# print(df.isnull().sum()) # It's ok

In [64]:
# Eliminate rows with missing values
# df.dropna(inplace=True) # It's ok

# Selección de datos útiles
## Para la matriz de datos de análisis 'X' se seleccionan todos los datos excepto los siguientes: 
* **tier**: Liga, dado que nuestra variable objetivo
* **rank**: Dado que es una variable que subdivide las ligas, no es de interés para este caso.
* **tierRank**: Fusión de las dos variables anteriores.
* **summonerName**: Nombre del jugador, irrelevante.
* **lane**: Posición del jugador en el mapa del juego al inicio del juego, irrelevante.
* **role**: Rol del jugador durante el juego, irrelevante.
## Estos se descartan debido a que no entregan información relevante en la partida de los jugadores.

In [65]:
X = df.drop(['region', 'summonerName', 'lane', 'role', 'championId'], axis=1).values
X_columns = df.drop(['region', 'summonerName', 'lane', 'role', 'championId'], axis=1).columns
y = df[['region']].to_numpy().ravel()
X_columns

Index(['gameId', 'tier', 'rank', 'tierRank', 'wins', 'losses', 'win',
       'spell1Id', 'spell2Id', 'kills', 'deaths', 'assists',
       'largestKillingSpree', 'largestMultiKill', 'killingSprees',
       'longestTimeSpentLiving', 'doubleKills', 'tripleKills', 'quadraKills',
       'pentaKills', 'totalDamageDealt', 'totalDamageDealtToChampions',
       'totalHeal', 'totalUnitsHealed', 'damageDealtToObjectives',
       'timeCCingOthers', 'totalDamageTaken', 'totalMinionsKilled',
       'goldEarned', 'goldSpent', 'visionScore', 'team-firstBlood',
       'team-firstTower', 'team-firstInhibitor', 'team-firstBaron',
       'team-firstDragon', 'team-firstRiftHerald', 'team-towerKills',
       'team-inhibitorKills', 'team-baronKills', 'team-dragonKills',
       'team-vilemawKills', 'team-riftHeraldKills'],
      dtype='object')

In [66]:
# encoding categorical data e.g. tier as a dummy variable
labelencoder_X = LabelEncoder()
X[:,1] = labelencoder_X.fit_transform(X[:,1])
X[:,2] = labelencoder_X.fit_transform(X[:,2])
X[:,3] = labelencoder_X.fit_transform(X[:,3])

# encoding categorical data e.g. tier as a dummy variable
y,class_names = pd.factorize(y)
y = y.astype('bool')
y

array([False, False, False, ...,  True,  True,  True])

In [67]:
# Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, stratify=y, random_state = 42)

In [68]:
names = [
            # "Nearest Neighbors",
            # "SVM Linear",
            # "SVM rbf",
            # "GP",
            " Decision Tree ",
            " Random Forest ",
            # "Neural Net", 
            "== Ada Boost ==",
            # "Naive Bayes",
            "===== QDA ====="
        ]        
        
classifiers = [
    # KNeighborsClassifier(500),
    # SVC(kernel="linear", C=0.03, probability = True),
    # SVC(gamma=1.5, C=1, probability = True),
    # GaussianProcessClassifier(1.0 * RBF(1.0), warm_start=True),
    DecisionTreeClassifier(criterion='gini', max_depth=10, random_state=42),
    RandomForestClassifier(criterion='gini', max_depth=10, random_state=42),
    # MLPClassifier(alpha=1),
    AdaBoostClassifier(),
    # GaussianNB(),
    QuadraticDiscriminantAnalysis()
    ]

In [69]:
i=1
fitted_clfs=[]
for name, clf in zip(names, classifiers):
        # fit the model using the training set
        fitted_clfs.append(clf.fit(X_train, y_train))   

C:\Users\matia\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:715: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


In [71]:
for name, clf in zip(names, fitted_clfs):
        y_pred = clf.predict(X_train)
        y_pred_labeled, y_train_labeled = class_names[y_pred.astype('int')], class_names[y_train.astype('int')]
        print('========================'+ name + '========================')
        print("Confusion matrix:")
        print(pd.crosstab(y_train_labeled, y_pred_labeled, rownames=['Actual'], colnames=['Predicted']))
        print("Classification report:")
        print(classification_report(y_train, y_pred, target_names=class_names))
        print('===============================================================\n')
        print(' ')
        i += 1

======================== Decision Tree ========================
Confusion matrix:
Predicted    br1     kr
Actual                 
br1        22257      0
kr             0  21789
Classification report:
              precision    recall  f1-score   support

         br1       1.00      1.00      1.00     22257
          kr       1.00      1.00      1.00     21789

    accuracy                           1.00     44046
   macro avg       1.00      1.00      1.00     44046
weighted avg       1.00      1.00      1.00     44046


 
======================== Random Forest ========================
Confusion matrix:
Predicted    br1     kr
Actual                 
br1        22257      0
kr             0  21789
Classification report:
              precision    recall  f1-score   support

         br1       1.00      1.00      1.00     22257
          kr       1.00      1.00      1.00     21789

    accuracy                           1.00     44046
   macro avg       1.00      1.00      1.00     44

In [ ]:
for name, clf in zip(names, fitted_clfs):
        y_pred = clf.predict(X_test)
        y_pred_labeled, y_test_labeled = class_names[y_pred.astype('int')], class_names[y_test.astype('int')]
        print('========================'+ name + '========================')
        print("Confusion matrix:")
        print(pd.crosstab(y_test_labeled, y_pred_labeled, rownames=['Actual'], colnames=['Predicted']))
        print("Classification report:")
        print(classification_report(y_test, y_pred, target_names=class_names))
        print('===============================================================\n')
        print(' ')
        i += 1

Estos primeros resultados, se puede apreciar que el clasificador que mejor funciona es Decision Tree, esto se debe a que............
Seguidamente el 2do mejor es Random Forest ........
Se hicieron diferentes pruebas cambiando los parametros de los clasificadores, como por ejemplo en decision tree se cambio repetidas veces el parametro de "random_state" debido a que nos entregaba una mejor clasificación y se queria comprobar si ese parametro era decisivo al momento de clasificar las ligas de los jugadores. 

In [ ]:
df.iloc[25000]

gameId                          2310762636
region                                 br1
summonerName                           ü u
tier                               DIAMOND
rank                                    II
tierRank                        DIAMOND-II
wins                                   138
losses                                 133
win                                   True
lane                                BOTTOM
role                           DUO_SUPPORT
championId                              43
spell1Id                                 4
spell2Id                                 7
kills                                    7
deaths                                   3
assists                                 18
largestKillingSpree                      3
largestMultiKill                         2
killingSprees                            2
longestTimeSpentLiving                 337
doubleKills                              1
tripleKills                              0
quadraKills

In [ ]:
df.iloc[55000]


gameId                         5344125983
region                                 kr
summonerName                        뚜뚜친구들
tier                              DIAMOND
rank                                   II
tierRank                       DIAMOND-II
wins                                  161
losses                                145
win                                  True
lane                               JUNGLE
role                                 NONE
championId                            234
spell1Id                               11
spell2Id                               54
kills                                  24
deaths                                 12
assists                                 7
largestKillingSpree                     7
largestMultiKill                        3
killingSprees                           6
longestTimeSpentLiving                272
doubleKills                             4
tripleKills                             1
quadraKills                       

Como conclusiónes podemos determinar que los datos entregados por la API de riot games son suficientes para poder clasificar al jugador con respecto a su liga, esto se debe que a nivel de juego los jugadores de ligas más altas toman distintas decisiones y cumplen diferentes objetivos. Una de las decisiones más notorias al momento de clasificar es la cantidad de minions que consiguen los jugadores de ligas más altas como por ejemplo en diamante que podemos notar que hizo un total de 171 minions vs un iron que hizo 22 en total, estos son elementos importantes debido a que los hace ganar más oro lo que permite que se compren items que les  mejoran sus habilidades como jugador. 



In [ ]:
for name, clf in zip(names, fitted_clfs):
        try:
                y_pred = clf.predict(X_test)
                y_pred_labeled, y_test_labeled = class_names[y_pred], class_names[y_test]
                print(name)
                feat_datas = sorted(list(zip(X_columns, clf.feature_importances_)), key=lambda x: x[1])
                for feat_data in feat_datas:
                        if feat_data[1] > 0.0005:
                                print(feat_data)
                print(' ')
        except:
                print('F')


 Decision Tree 
('gameId', 1.0)
 
 Random Forest 
('wins', 0.1)
('spell1Id', 0.1)
('deaths', 0.1)
('goldEarned', 0.1)
('team-dragonKills', 0.1)
('spell2Id', 0.2)
('losses', 0.3)
 
== Ada Boost ==
('gameId', 1.0)
 
===== QDA =====
F


In [ ]:
df[df['summonerName'] == '뚜뚜친구들'].describe()


,gameId,wins,losses,championId,spell1Id,spell2Id,kills,deaths,assists,largestKillingSpree,...,totalMinionsKilled,goldEarned,goldSpent,visionScore,team-towerKills,team-inhibitorKills,team-baronKills,team-dragonKills,team-vilemawKills,team-riftHeraldKills
count,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# # Visualize the tree by graphiz
# import graphviz
# from sklearn import tree
# import os
# os.environ["PATH"] += os.pathsep + 'C:/Users/matia/anaconda3/Library/bin/graphviz/'
# feature_names = X_columns
# dot_data = tree.export_graphviz(classifier, out_file=None, filled=True, rounded = True, feature_names=feature_names, class_names=class_names)
# graph = graphviz.Source(dot_data)
# graph